In [1]:
#
# Non parallel
#
from numba import jit, njit, cuda , prange
import numba
import numpy as np

# registers = [729, 0, 0]
# program = [0,1,5,4,3,0]

register_a = 0
register_b = 1
register_c = 2

# enabling the JIT causes print statements to not print lists as expected
# @njit(parallel=True)
def loop_em(start, end, increment, program):
    # This program is 8 octal operands so the A register needs to be 8 octal digits.
    # mutable so we will copy
    registers_orig = np.array([52884621, 0, 0])
    print(program)

    loop_iterations = (end-start) // increment
    print(f'total size ={loop_iterations} program = {program}')

    for i in prange(loop_iterations):
        current = (i*increment) + start
        registers = registers_orig.copy()
        registers[0] = current
        output = []
        resolve_operand = lambda operand: operand if (operand<4) else registers[operand-4]
        # if (i%1000000 == 0):
        #     print(f'at: {current} : {i}/{loop_iterations}')
        
        # print(f'{i}:{current}:{end} program: {program} registers: {registers}')
        address_ptr = 0
        while (address_ptr < len(program)):
            # numba says these are int64
            operator = program[address_ptr]
            operand = program[address_ptr+1]
            next_address_ptr = address_ptr+2
            # print (f'address: {address_ptr} operator: {operator} operand: {operand} registers: {registers} ')
            match (operator):
                case 0: # adv division register_a ~/ 2^comboOperand
                    registers[register_a] = registers[register_a] // 2 ** resolve_operand(operand)
                case 1: # bxl bitwise XOR (registerB , operand)
                    registers[register_b] = registers[register_b] ^ operand
                case 2: # bst operand modulo 8
                    registers[register_b] = resolve_operand(operand) % 8
                case 3: # jnz jump not zero
                    if (registers[register_a] != 0):
                        next_address_ptr =  operand
                case 4: #bxc bitwise xor reg b, reg c
                    registers[register_b] = registers[register_b] ^ registers[register_c]
                case 5: # out % modulo 8
                    output.append(resolve_operand(operand) %8)
                case 6: # BDV integer division on A , stored in B
                    divisor = 2 ** resolve_operand(operand)
                    registers[register_b] = registers[register_a] // divisor
                case 7: # CDV
                    divisor = 2 ** resolve_operand(operand)
                    registers[register_c] = registers[register_a] // divisor
                case _:
                    print('oh no')
                    result = -1
            
            address_ptr = next_address_ptr
            # print(f'now at: {address_ptr} output after {output}')
        # print(f'final registers: {registers} output {np.array(output)} ')
        if (len(output) >= 6
            and output[0]==program[0] 
            and output[1]==program[1] 
            and output[2]==program[2] 
            and output[3]==program[3]
            and output[4]==program[4]
            and output[5]==program[5]
            # and output[6]==program[6]
            # and output[7]==program[7]
            # and output[8]==program[8]
            # and output[9]==program[9]
            # and output[10]==program[10]
            # and output[11]==program[11]
            # and output[12]==program[12]
           ):
            # will not print if njit is enabled
            #print(f'matches {oct(current)} - {output} -{current}')
            # use with njit
            print(f'matches {current}')
            print(output)
            # from before we put the loop in
            # return current
        # print(f'{oct(current)} - {output}')
        # return output
    print('done');


In [ ]:
%%time
loop_em(int(0o1000000000000000),
        int(0o1000077777777777),
        int(0o0000000000000001),
        np.array([2,4,1,3,7,5,4,7,0,3,1,5,5,5,3,0]))

# 16 digits octal
# 16th digit must be 1 otherwise the return is shorter than the program


[2 4 1 3 7 5 4 7 0 3 1 5 5 5 3 0]
total size =8589934591 program = [2 4 1 3 7 5 4 7 0 3 1 5 5 5 3 0]
